In [ ]:
# Copyright 2025 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG Engine in Vertex AI


In [1]:
# @title Install packages
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,adk,langchain,ag2,llama_index] \
                                      google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai

#### Note: <font color="red">Need to restart the kernel after installing</font>

In [3]:
exit()

In [1]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

In [2]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [3]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [4]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://agent-0417/
No changes made to gs://agent-0417/


In [6]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai
from vertexai import agent_engines
from google import genai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

# client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [7]:
# @title Create a RAG Corpus
# Currently supports Google first-party embedding models
# https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#supported-models

from vertexai import rag

EMBEDDING_MODEL="publishers/google/models/text-multilingual-embedding-002"

rag_corpus = rag.create_corpus(
    display_name="it-laws",
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=rag.RagEmbeddingModelConfig(
            vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
                publisher_model=EMBEDDING_MODEL
            )
        )
    ),
)

### Check the corpus just created

In [8]:
rag.list_corpora()

ListRagCorporaPager<rag_corpora {
  name: "projects/ai-hangsik/locations/us-central1/ragCorpora/7186056155423047680"
  display_name: "ai_agent"
  create_time {
    seconds: 1744176924
    nanos: 867152000
  }
  update_time {
    seconds: 1744176924
    nanos: 867152000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/ai-hangsik/locations/us-central1/publishers/google/models/text-multilingual-embedding-002"
      }
    }
  }
}
rag_corpora {
  name: "projects/ai-hangsik/locations/us-central1/ragCorpora/2842897264777625600"
  display_name: "Alphabet_10K_2024_corpus"
  description: "Corpus containing Alphabet\'s 10-K 2024 document"
  create_time {
    seconds: 1744716243
    nanos: 402169000
  }
  update_time {
    seconds: 1744716243
    nanos: 402169000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
    }
    rag

In [9]:
# @title Import files from Google Cloud Storage

transformation_config = rag.TransformationConfig(
      chunking_config=rag.ChunkingConfig(
          chunk_size=512,
          chunk_overlap=100,
      ),
  )

# INPUT_GCS_BUCKET = ["gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf", "gs://it_laws_kr/law_pdf/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf",]
INPUT_GCS_BUCKET = "gs://it_laws_kr/law_pdf/"

response = rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[INPUT_GCS_BUCKET],
    transformation_config=transformation_config, # Optional
    max_embedding_requests_per_min=1000,  # Optional
)

In [ ]:
rag.list_files(rag_corpus.name)

ListRagFilesPager<rag_files {
  name: "projects/721521243942/locations/us-central1/ragCorpora/4572279521687896064/ragFiles/5413701574923261104"
  display_name: "개인정보 보호법(법률)(제19234호)(20240315).pdf"
  create_time {
    seconds: 1744875147
    nanos: 73110000
  }
  update_time {
    seconds: 1744875147
    nanos: 73110000
  }
  gcs_source {
    uris: "gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf"
  }
  file_status {
    state: ACTIVE
  }
}
rag_files {
  name: "projects/721521243942/locations/us-central1/ragCorpora/4572279521687896064/ragFiles/5413701623427508934"
  display_name: "신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
  create_time {
    seconds: 1744875153
    nanos: 199117000
  }
  update_time {
    seconds: 1744875153
    nanos: 199117000
  }
  gcs_source {
    uris: "gs://it_laws_kr/law_pdf/신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
  }
  file_status {
    state:

### Perform direct context retrieval

In [ ]:
rag_retrieval_config=rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5)  # Optional
)

# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="개인정보 보호법에 대해서 설명해주세요.",
    rag_retrieval_config=rag_retrieval_config,
)
print(response)

# context = " ".join([context.text for context in response.contexts.contexts]).replace("\n", "")

contexts {
  contexts {
    source_uri: "gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf"
    text: "법제처 1 국가법령정보센터\r\n개인정보 보호법\r\n \r\n개인정보 보호법\r\n[시행 2024. 3. 15.] [법률 제19234호, 2023. 3. 14., 일부개정]\r\n개인정보보호위원회 (개인정보보호정책과 - 법령 제개정) 02-2100-3057\r\n개인정보보호위원회 (심사총괄담당관 - 법령 해석) 02-2100-3043\r\n 제1장 총칙\r\n제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인\r\n의 존엄과 가치를 구현함을 목적으로 한다. <개정 2014. 3. 24.>\r\n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014. 3. 24., 2020. 2. 4., 2023. 3. 14.>\r\n1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다.\r\n가."
    source_display_name: "개인정보 보호법(법률)(제19234호)(20240315).pdf"
    score: 0.15580536827091096
  }
  contexts {
    source_uri: "gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf"
    text: "<개정 2023. 3. 14.>\r\n1. 개인정보의 수집 출처법제처 10 국가법령정보센터\r\n개인정보 보호법\r\n2. 개인정보의 처리 목적\r\n3. 제37조에 따른 개인정보 처리의 정지를 요구하거나 동의를 철회할 권리가 있다는 사실\r\n② 제1항에도 불구하고 처리하는 개인정보의 종류ㆍ규모, 종업원 수 및

### Create RAG Retrieval Tool

In [ ]:
# Create a tool for the RAG Corpus
from vertexai.generative_models import GenerativeModel, Tool

rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            rag_retrieval_config=rag_retrieval_config,
        ),
    )
)

## Use RAG Engine on Agent Engine.

In [ ]:
# @title Agent Define - Local unit test
# from vertexai.preview.reasoning_engines import LangchainAgent
from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.LangchainAgent

local_agent = agent_engines.LangchainAgent(
    model = MODEL_NAME,
    model_kwargs={
      "temperature": 0.28,
      "max_output_tokens": 1000,
      "top_p": 0.95,
      "top_k": None,
    },
    tools = [rag_retrieval_tool],
    # agent_executor_kwargs={"return_intermediate_steps": False},
)

In [ ]:
# @title Test your agent locally
# Agent local unit test before deploying.
local_agent.query(input="개인정보 보호법에 대해서 설명해주세요.")

{'input': '개인정보 보호법에 대해서 설명해주세요.',
 'output': '개인정보 보호법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인의 존엄과 가치를 구현함을 목적으로 합니다. 이 법은 2024년 3월 15일에 일부 개정되어 시행되고 있습니다.\n\n주요 내용으로는 개인정보의 정의, 개인정보 보호 원칙, 개인정보 보호위원회의 설치 및 운영, 다른 법률과의 관계 등이 있습니다. 개인정보 보호위원회는 국무총리 소속으로 개인정보 보호에 관한 사무를 독립적으로 수행합니다. 또한, 개인정보 처리자는 정보주체 이외로부터 개인정보를 수집하여 처리하는 경우, 정보주체에게 일정한 사항을 알려야 할 의무가 있습니다. 국가와 지방자치단체는 개인정보를 처리할 때 정보주체의 권리가 보장될 수 있도록 개인정보 보호 원칙에 맞게 적용해야 합니다.'}

## Use previous way to call Gemini model.

In [ ]:
# @title Generate Content with Gemini using Rag Retrieval Tool
# Load tool into Gemini model
rag_gemini_model = GenerativeModel(
    MODEL_NAME,
    tools=[rag_retrieval_tool],
)

In [ ]:
response = rag_gemini_model.generate_content("개인정보 보호법에 대해서 설명해주세요.")
display(Markdown(response.text))

개인정보 보호법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인의 존엄과 가치를 구현함을 목적으로 합니다.

주요 내용으로는 개인정보 보호 원칙, 개인정보 보호위원회의 설치, 그리고 다른 법률과의 관계 등이 있습니다. 개인정보 보호위원회는 국무총리 소속으로 개인정보 보호에 관한 사무를 독립적으로 수행합니다. 또한, 개인정보의 처리 및 보호에 관하여 다른 법률에 특별한 규정이 있는 경우를 제외하고는 이 법에서 정하는 바에 따르며, 다른 법률을 제정하거나 개정하는 경우에는 이 법의 목적과 원칙에 맞도록 해야 합니다.

개인정보 보호법은 2024년 3월 15일에 일부 개정되어 시행되고 있습니다.